<a id="top"></a>
# 05 · Resumen y selección de resultados (continual)

**Qué hace este notebook**

A partir de las carpetas `outputs/continual_*`:

- Construye una tabla consolidada de runs (`results_table_fromdisk.csv`) con:
  - preset, método, encoder, modelo, seed, T, batch_size, amp,
  - MAE por tarea (best/final),
  - métricas de olvido (abs/rel),
  - emisiones y tiempo (si hay CodeCarbon / telemetry).
- Selecciona un subconjunto **comparable** de runs (`selection_table.csv`).
- Calcula:
  - un **ranking compuesto** rendimiento+olvido y los *winners* por método  
    (`winners_per_methodbase.csv` o `winners_per_fullmethod.csv`),
  - tabla relativa frente a EWC (`relative_to_ewc.csv`) y *scorecards* por método  
    (`scorecards_por_metodo_vs_ewc.csv`),
  - Top-6 global (`top6_composite.csv`) y frente de Pareto (`pareto.csv`),
  - Top-5 por método con hiperparámetros parseados (`top5_por_metodo.csv` + `.tex`).

> Este notebook **no entrena nada**: solo lee ficheros de `outputs/`.

**Prerrequisitos**

- Haber ejecutado **los runners continual** que crean carpetas `outputs/continual_*`.
- (Opcional) Haber activado CodeCarbon para obtener `emissions.csv`.

---

<a id="toc"></a>
## Índice

- [1) Configuración y filtros globales](#sec-01)
- [2) Utilidades de lectura y parseo](#sec-02)
- [3) Tabla de resultados desde disco (`results_table_fromdisk.csv`)](#sec-03)
- [4) Selección de runs comparables (`selection_table.csv`)](#sec-04)
- [5) Ranking compuesto y *winners* por método](#sec-05)
- [6) Comparativa relativa frente a EWC](#sec-06)
- [7) Top-6 + frente de Pareto y *scatter* MAE vs olvido](#sec-07)
- [8) Top-5 por método (tabla + LaTeX)](#sec-08)
- [9) Resumen de artefactos generados](#sec-09)


<a id="sec-01"></a>
## 1) Configuración y filtros globales

**Objetivo**

- Detectar la **raíz de outputs** (`OUT`).
- Definir filtros por:
  - `preset` (`PRESET_KEEP`),
  - `encoder` (`ENCODER_KEEP`),
  - `seed`, `method`,
  - fecha mínima de modificación (`SINCE`).
- Configurar opciones de comparabilidad y del *score* compuesto.

> Puedes sobreescribir los filtros con variables de entorno:
> `PRESET_KEEP`, `ENCODER_KEEP`, `SEED_KEEP`, `METHODS_KEEP`, `SINCE`, etc.

[↑ Volver al índice](#toc)


In [30]:
from pathlib import Path
import pandas as pd
import numpy as np
import math, re, os, json
from datetime import datetime

# -------- Raíz de outputs del proyecto --------
OUT = Path(os.environ.get("OUT_DIR", "/home/cesar/proyectos/TFM_SNN/outputs")).resolve()

# -------- Filtros "paper set" (por defecto) --------
def _parse_set(env_name, default):
    val = os.environ.get(env_name, None)
    if val is None:
        return default
    if val.strip().lower() in {"none", "", "all"}:
        return None
    return set([v.strip() for v in val.split(",") if v.strip()])

# Por defecto: preset std + encoder rate
PRESET_KEEP   = _parse_set("PRESET_KEEP",   {"std"})   # None → no filtra por preset
ENCODER_KEEP  = _parse_set("ENCODER_KEEP",  {"rate"})  # None → no filtra
SEED_KEEP     = _parse_set("SEED_KEEP",     None)      # p.ej. "42,7"
METHODS_KEEP  = _parse_set("METHODS_KEEP",  None)

ONLY_NEW_RUNNER = os.environ.get("ONLY_NEW_RUNNER", "1") not in {"0", "false", "False"}

# Ventana temporal opcional: SINCE="2025-11-11" o SINCE="3d" (últimos 3 días).
# SINCE = os.environ.get("SINCE", "").strip()
# UNTIL = os.environ.get("UNTIL", "").strip()
SINCE="2025-11-11"
UNTIL ="2025-12-04"
# SINCE="2025-12-06"
# UNTIL ="2025-12-11"

def _compute_ts(spec: str):
    if not spec:
        return None
    try:
        # Formato "3d" → últimos 3 días
        if spec.lower().endswith("d"):
            days = int(spec[:-1])
            return pd.Timestamp.now().normalize() - pd.Timedelta(days=days)
        # Formato fecha "YYYY-MM-DD"
        return pd.Timestamp(spec)
    except Exception:
        return None

MTIME_FROM = _compute_ts(SINCE)
MTIME_TO   = _compute_ts(UNTIL)

# -------- Comparabilidad "dura" --------
STRICT_CFG = os.environ.get("STRICT_CFG", "1") not in {"0", "false", "False"}

# -------- Métrica compuesta (trade-off rendimiento/olvido) --------
ALPHA_COMPOSITE = float(os.environ.get("ALPHA_COMPOSITE", "0.5"))

# -------- Opciones de informe / narrativa --------
GROUP_BY_FULL_METHOD   = os.environ.get("GROUP_BY_FULL_METHOD", "0") in {"1","true","True"}
IGNORE_NAIVE_IN_REPORTS = True      # excluye naive de tablas/plots de informe
RELATIVE_BASELINE       = "ewc"     # baseline para comparativas (None para desactivar)
BASELINE_MATCH_STRICT   = True      # emparejamiento estricto de runs comparables

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 180)

# -------- Carpeta SUMMARY dinámica --------
def _token(val):
    if val is None: return "all"
    if isinstance(val, (set, list, tuple)):
        return "-".join(sorted(map(str, val)))
    return str(val)

now = pd.Timestamp.now(tz="Europe/Madrid")
auto_label = f"paperset_{_token(PRESET_KEEP)}_{_token(ENCODER_KEEP)}_{now.strftime('%Y-%m-%d_%H%M')}"
SUMMARY_LABEL = os.environ.get("SUMMARY_LABEL", "").strip() or auto_label

SUMMARY = (OUT / "summary" / SUMMARY_LABEL).resolve()
SUMMARY.mkdir(parents=True, exist_ok=True)
print(f"[SUMMARY] {SUMMARY}")

# Symlink "latest" → esta ejecución (si el FS lo permite)
LATEST = OUT / "summary" / "latest"
try:
    if LATEST.exists() or LATEST.is_symlink():
        LATEST.unlink()
    LATEST.symlink_to(SUMMARY, target_is_directory=True)
except Exception:
    pass

# Manifest de filtros/ajustes para trazabilidad
manifest = {
    "summary_label": SUMMARY_LABEL,
    "created_at": now.isoformat(),
    "OUT": str(OUT),
    "filters": {
        "PRESET_KEEP": sorted(list(PRESET_KEEP)) if PRESET_KEEP else None,
        "ENCODER_KEEP": sorted(list(ENCODER_KEEP)) if ENCODER_KEEP else None,
        "SEED_KEEP": sorted(list(SEED_KEEP)) if SEED_KEEP else None,
        "METHODS_KEEP": sorted(list(METHODS_KEEP)) if METHODS_KEEP else None,
        "ONLY_NEW_RUNNER": ONLY_NEW_RUNNER,
        "MTIME_FROM": str(MTIME_FROM) if MTIME_FROM is not None else None,
    },
    "comparability": {"STRICT_CFG": STRICT_CFG},
    "composite": {"ALPHA_COMPOSITE": ALPHA_COMPOSITE},
    "report_opts": {
        "IGNORE_NAIVE_IN_REPORTS": IGNORE_NAIVE_IN_REPORTS,
        "GROUP_BY_FULL_METHOD": GROUP_BY_FULL_METHOD,
        "RELATIVE_BASELINE": RELATIVE_BASELINE,
        "BASELINE_MATCH_STRICT": BASELINE_MATCH_STRICT,
    },
}
(SUMMARY / "summary_manifest.json").write_text(json.dumps(manifest, indent=2), encoding="utf-8")

# --- Manifest "último" para reutilizar rutas en otras celdas/notebooks
LAST_PATH = OUT / "summary" / "_last.json"

def _last_update(**kwargs):
    # Carga estado actual (si existe)
    try:
        cur = json.loads(LAST_PATH.read_text(encoding="utf-8"))
    except Exception:
        cur = {}

    cur.update({
        "summary_label": SUMMARY_LABEL,
        "summary_dir": str(SUMMARY.resolve()),
        "created_at": now.isoformat(),
    })

    # Normaliza nuevas rutas/etiquetas
    for k, v in list(kwargs.items()):
        if isinstance(v, (str, Path)):
            kwargs[k] = str(Path(v).resolve())
    cur.update(kwargs)

    LAST_PATH.write_text(json.dumps(cur, indent=2), encoding="utf-8")
    print(f"[LAST] actualizado → {LAST_PATH}")

# Inicializamos el manifest _last.json
_last_update()

[SUMMARY] /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_std_rate_2025-12-11_2042
[LAST] actualizado → /home/cesar/proyectos/TFM_SNN/outputs/summary/_last.json


<a id="sec-02"></a>
## 2) Utilidades de lectura y parseo

**Objetivo**

Reunir utilidades canónicas para:

- lectura robusta de JSON/CSV (`_read_json`, `_read_csv_df`),
- normalización de métodos (`canonical_method`),
- cálculo de tiempos de modificación (`run_mtime`),
- lectura de olvido (`_read_forgetting`).

> Estas funciones se reutilizan en la construcción de la tabla de resultados.

[↑ Volver al índice](#toc)


In [31]:
from pathlib import Path

def _safe_float(x, default=np.nan):
    try:
        if x is None: return default
        if isinstance(x, (int, float)): return float(x)
        s = str(x).strip()
        if s.lower() in {"nan","none","null",""}: return default
        return float(s)
    except Exception:
        return default

def _read_json(p: Path):
    try:
        if p.exists():
            return json.loads(p.read_text(encoding="utf-8"))
    except Exception:
        pass
    return None

def _read_csv_df(p: Path):
    try:
        if p.exists():
            return pd.read_csv(p)
    except Exception:
        pass
    return None

def _abs_run_dir(rel: str|Path) -> Path:
    rel = str(rel)
    return (OUT / rel) if not rel.startswith(str(OUT)) else Path(rel)

def run_mtime(rel: str|Path) -> float:
    rd = _abs_run_dir(rel)
    mt = 0.0
    for root, _, files in os.walk(rd):
        for f in files:
            try:
                mt = max(mt, (Path(root)/f).stat().st_mtime)
            except Exception:
                pass
    return mt

def canonical_method(m: str) -> str:
    if not m: return "none"
    m = m.lower()
    if m.startswith("ewc"):       return "ewc"
    if m.startswith("as-snn"):    return "as-snn"
    if m.startswith("sa-snn"):    return "sa-snn"
    if m.startswith("sca-snn"):   return "sca-snn"
    if m.startswith("rehearsal"): return "rehearsal"
    if m.startswith("naive"):     return "naive"
    return m

def _read_forgetting(run_dir: Path) -> dict:
    """
    Lee primero forgetting_summary.json (preferente).
    Si no existe, lee forgetting.json (anidado por tareas) y calcula la media.
    Devuelve claves normalizadas:
      circuito1_forget_abs/rel, circuito2_forget_abs/rel, avg_forget_rel.
    """
    def _nanmean2(a, b):
        vals = [x for x in [a, b] if x is not None and not math.isnan(_safe_float(x))]
        return float(np.mean([_safe_float(x) for x in vals])) if vals else np.nan

    # 1) summary (preferido)
    summ = _read_json(run_dir / "forgetting_summary.json")
    if isinstance(summ, dict):
        c1_abs = _safe_float(summ.get("circuito1_forget_abs", summ.get("task_1_forget_abs")))
        c1_rel = _safe_float(summ.get("circuito1_forget_rel", summ.get("task_1_forget_rel")))
        c2_abs = _safe_float(summ.get("circuito2_forget_abs", summ.get("task_2_forget_abs")))
        c2_rel = _safe_float(summ.get("circuito2_forget_rel", summ.get("task_2_forget_rel")))
        avg_rel = _safe_float(summ.get("avg_forget_rel", summ.get("avg_forgetting_rel")))
        if math.isnan(avg_rel):
            avg_rel = _nanmean2(c1_rel, c2_rel)
        return {
            "circuito1_forget_abs": c1_abs,
            "circuito1_forget_rel": c1_rel,
            "circuito2_forget_abs": c2_abs,
            "circuito2_forget_rel": c2_rel,
            "avg_forget_rel":       avg_rel,
        }

    # 2) forgetting.json (anidado por tareas) o plano
    js = _read_json(run_dir / "forgetting.json") or {}
    if not isinstance(js, dict):
        return {}

    def pick(d, *keys):
        for k in keys:
            if k in d: return d[k]
        return None

    c1_abs = _safe_float(pick(js,
                              "circuito1_forget_abs","task_1_circuito1_forget_abs",
                              "c1_forget_abs","task_1_forget_abs"))
    c1_rel = _safe_float(pick(js,
                              "circuito1_forget_rel","task_1_circuito1_forget_rel",
                              "c1_forget_rel","task_1_forget_rel"))
    c2_abs = _safe_float(pick(js,
                              "circuito2_forget_abs","task_2_circuito2_forget_abs",
                              "c2_forget_abs","task_2_forget_abs"))
    c2_rel = _safe_float(pick(js,
                              "circuito2_forget_rel","task_2_circuito2_forget_rel",
                              "c2_forget_rel","task_2_forget_rel"))
    avg_rel = _safe_float(pick(js,
                               "avg_forget_rel","avg_forgetting_rel",
                               "mean_forget_rel","forget_rel_avg"))

    # Si sigue faltando, leer anidado por tarea
    if ("circuito1" in js) and isinstance(js["circuito1"], dict):
        if math.isnan(c1_abs): c1_abs = _safe_float(js["circuito1"].get("forget_abs"))
        if math.isnan(c1_rel): c1_rel = _safe_float(js["circuito1"].get("forget_rel"))
    if ("circuito2" in js) and isinstance(js["circuito2"], dict):
        if math.isnan(c2_abs): c2_abs = _safe_float(js["circuito2"].get("forget_abs"))
        if math.isnan(c2_rel): c2_rel = _safe_float(js["circuito2"].get("forget_rel"))

    if math.isnan(avg_rel):
        avg_rel = _nanmean2(c1_rel, c2_rel)

    return {
        "circuito1_forget_abs": c1_abs,
        "circuito1_forget_rel": c1_rel,
        "circuito2_forget_abs": c2_abs,
        "circuito2_forget_rel": c2_rel,
        "avg_forget_rel":       avg_rel,
    }

<a id="sec-03"></a>
## 3) Tabla de resultados desde disco (`results_table_fromdisk.csv`)

**Objetivo**

Reconstruir una tabla consolidada de runs:

- Una fila por carpeta `outputs/continual_*`.
- MAEs por tarea (best/final), olvido, emisiones, tiempo, metadatos.
- Exportar a `SUMMARY/results_table_fromdisk.csv`.

> Esta tabla es la base del resto de análisis.

[↑ Volver al índice](#toc)


In [32]:
import os, re, json, math
import numpy as np
import pandas as pd
from pathlib import Path

def _parse_basic_meta(run_dir: Path) -> dict:
    """
    Extrae preset, method, encoder, model, seed, T, amp, batch_size desde los artefactos.
    Prioridad: run_row.json → manifest de tarea → heurística de nombre de carpeta.
    """
    jrow = _read_json(run_dir / "run_row.json") or {}

    def gj(*ks, default=None):
        obj = jrow
        for k in ks:
            if not isinstance(obj, dict) or (k not in obj):
                return default
            obj = obj[k]
        return obj

    preset   = jrow.get("preset")   or gj("meta","preset")
    method   = jrow.get("method")   or gj("meta","method")
    encoder  = jrow.get("encoder")  or gj("meta","encoder")
    model    = jrow.get("model")    or jrow.get("model_name") \
               or gj("meta","model") or gj("meta","model_name")
    seed     = jrow.get("seed")     or gj("meta","seed")
    T        = jrow.get("T")        or gj("data","T") or gj("meta","data","T") or gj("meta","T")
    amp      = jrow.get("amp")      or gj("training","amp") or gj("meta","amp")
    batch_sz = jrow.get("batch_size") or gj("meta","batch_size")

    # Manifest de la primera tarea (por si faltan cosas)
    man1 = _read_json(run_dir / "task_1_circuito1" / "manifest.json")
    if isinstance(man1, dict):
        meta1 = man1.get("meta", {}) if isinstance(man1.get("meta", {}), dict) else {}
        if not model:        model    = meta1.get("model") or man1.get("model") or man1.get("model_name")
        if batch_sz is None: batch_sz = meta1.get("batch_size", batch_sz)
        if T is None:        T        = meta1.get("T", T)
        if amp is None:      amp      = meta1.get("amp", amp)
        if not encoder:      encoder  = meta1.get("encoder", encoder)
        if not preset:       preset   = meta1.get("preset", preset)
        if not method:       method   = meta1.get("method", method)

    # Heurística de nombre de carpeta si faltan preset/method/encoder
    if not preset or not method or not encoder:
        name = run_dir.name
        m = re.match(r"continual_([^_]+)_([^_].*?)_(rate|latency|raw|image).*", name)
        if m:
            preset  = preset  or m.group(1)
            method  = method  or m.group(2)
            encoder = encoder or m.group(3)

    # Tipados/normalizaciones
    seed = _safe_float(seed)
    T    = _safe_float(T)
    if isinstance(amp, str):
        amp = amp.strip().lower() in {"true","1","yes","y"}
    elif isinstance(amp, (int, float)):
        amp = bool(amp)
    elif amp is not None and not isinstance(amp, bool):
        amp = None
    batch_sz = _safe_float(batch_sz)

    return dict(
        preset=preset, method=method, encoder=encoder, model=model,
        seed=seed, T=T, amp=amp, batch_size=batch_sz
    )

def _read_per_task_perf(run_dir: Path) -> dict:
    """
    Devuelve dict por tarea con {'best_mae','final_mae'} normalizadas.
    Soporta per_task_perf.json (lista/dict) y per_task_perf.csv.
    """
    def _norm_row_dictlike(d):
        tname = (str(d.get("task_name") or d.get("task") or d.get("name") or "")).strip().lower()
        best_candidates  = ["best_mae","val_best_mae","best","mae_best","min_mae","test_mae"]
        final_candidates = ["final_mae","val_final_mae","val_last_mae","last_mae","mae_last","mae_final"]
        best  = next((d.get(k) for k in best_candidates  if k in d), None)
        final = next((d.get(k) for k in final_candidates if k in d), None)
        return tname, {"best_mae": _safe_float(best), "final_mae": _safe_float(final)}

    out = {}

    # 1) JSON
    js = _read_json(run_dir / "per_task_perf.json")
    if js is not None:
        if isinstance(js, list):
            for row in js:
                if isinstance(row, dict):
                    t, val = _norm_row_dictlike(row)
                    if t:
                        out[t] = val
        elif isinstance(js, dict):
            if all(isinstance(v, dict) for v in js.values()):
                for k, v in js.items():
                    t, val = _norm_row_dictlike({"task_name": k, **v})
                    if t:
                        out[t] = val
            else:
                try:
                    df = pd.DataFrame(js)
                    for _, row in df.iterrows():
                        t, val = _norm_row_dictlike(row.to_dict())
                        if t:
                            out[t] = val
                except Exception:
                    pass

    # 2) CSV (fusiona y rellena NaNs si los hay)
    df = _read_csv_df(run_dir / "per_task_perf.csv")
    if df is not None and not df.empty:
        for _, row in df.iterrows():
            tn = (str(row.get("task_name") or row.get("task") or row.get("name") or "")).strip().lower()
            if not tn:
                continue
            cur = out.get(tn, {"best_mae": np.nan, "final_mae": np.nan})
            if math.isnan(_safe_float(cur.get("best_mae"))):
                cur["best_mae"] = _safe_float(row.get("val_best_mae", row.get("best_mae", row.get("test_mae"))))
            if math.isnan(_safe_float(cur.get("final_mae"))):
                cur["final_mae"] = _safe_float(row.get("val_last_mae",
                                                       row.get("val_final_mae",
                                                               row.get("final_mae"))))
            out[tn] = cur
    return out

def _read_method_params(run_dir: Path) -> dict:
    """
    Carga method_params.json si existe.

    Soporta dos formas:
      - plano:   { "k": 8, "tau": 28, ... }
      - anidado: { "method": "sa-snn", "params": { ... } }

    Devuelve siempre un dict plano de hiperparámetros.
    """
    js = _read_json(run_dir / "method_params.json") or {}
    if not isinstance(js, dict):
        return {}
    if "params" in js and isinstance(js["params"], dict):
        return js["params"]
    # en caso contrario devolvemos el dict entero (puede estar ya plano)
    return js

def _read_efficiency(run_dir: Path):
    """Lee emisiones/tiempo desde efficiency_summary.json; si falta, intenta emissions.csv y run_row.json."""
    j = _read_json(run_dir / "efficiency_summary.json") or {}
    emissions = _safe_float(j.get("emissions_kg"), default=np.nan)
    elapsed   = _safe_float(j.get("elapsed_sec"),   default=np.nan)

    if math.isnan(emissions):
        df = _read_csv_df(run_dir / "emissions.csv")
        if df is not None:
            col = "co2e_kg" if "co2e_kg" in df.columns else ("emissions_kg" if "emissions_kg" in df.columns else None)
            if col:
                emissions = float(df[col].sum())

    if math.isnan(elapsed) or math.isnan(emissions):
        rj = _read_json(run_dir / "run_row.json") or {}
        if math.isnan(elapsed):
            elapsed = _safe_float(rj.get("elapsed_sec"), default=elapsed)
        if math.isnan(emissions):
            emissions = _safe_float(rj.get("emissions_kg"), default=emissions)

    # --- NUEVO: si el valor es sospechosamente pequeño, lo tratamos como "no disponible" ---
    # (no parece razonable que un entrenamiento entero de preset accurate consuma < 0.01 kg)
    if emissions is not None and math.isfinite(emissions) and emissions < 1e-2:
        emissions = np.nan

    return emissions, elapsed


def _read_eval_matrix(run_dir: Path) -> pd.DataFrame | None:
    """Carga eval_matrix (csv) o reconstruye desde json {'tasks':[], 'mae_matrix':[[]]}."""
    p_csv = run_dir / "eval_matrix.csv"
    if p_csv.exists():
        try:
            return pd.read_csv(p_csv)
        except Exception:
            pass

    js = _read_json(run_dir / "eval_matrix.json")
    if isinstance(js, dict) and ("tasks" in js) and ("mae_matrix" in js):
        tasks = list(js.get("tasks") or [])
        mat   = js.get("mae_matrix") or []
        try:
            mat = np.array(mat, dtype=float)
            cols = ["task"] + [f"after_{t}" for t in tasks]
            data = {"task": tasks}
            for j, cname in enumerate(cols[1:]):
                colvals = [row[j] if j < len(row) else np.nan for row in mat]
                data[cname] = colvals
            return pd.DataFrame(data)
        except Exception:
            try:
                return pd.DataFrame(js)
            except Exception:
                return None
    return None

def _compute_best_final_from_eval(eval_df: pd.DataFrame, task_token: str):
    """Obtiene BEST/FINAL mirando primero por filas (col 'task') y, si no existe, por columnas."""
    if eval_df is None or eval_df.empty:
        return (np.nan, np.nan)

    # 1) Formato fila: 'task' == task_token
    tcol = None
    for c in eval_df.columns:
        if str(c).strip().lower() == "task":
            tcol = c
            break
    if tcol is not None:
        mask = eval_df[tcol].astype(str).str.lower() == str(task_token).lower()
        if mask.any():
            row = eval_df.loc[mask].iloc[0]
            data_cols = [c for c in eval_df.columns if c != tcol]
            vals = pd.to_numeric(row[data_cols], errors="coerce").values.astype(float)
            finite = np.isfinite(vals)
            best = float(np.nanmin(vals)) if finite.any() else np.nan
            final = _safe_float(row[data_cols[-1]])
            return (best, final)

    # 2) Fallback: columnas que contengan el token
    cols = [c for c in eval_df.columns if str(task_token).lower() in str(c).lower()]
    if cols:
        dfc = eval_df[cols].apply(pd.to_numeric, errors="coerce")
        vals = dfc.values.astype(float)
        finite = np.isfinite(vals)
        best = float(np.nanmin(vals)) if finite.any() else np.nan
        final = _safe_float(dfc.iloc[-1, -1])
        return (best, final)

    return (np.nan, np.nan)

def _compute_forgetting_from_eval_matrix(eval_df: pd.DataFrame, per_task: dict):
    """Olvido para c1 tras aprender c2. C2 no olvida (0)."""
    out = {}
    t1_keys = [k for k in (per_task or {}).keys() if "circuito1" in str(k).lower()]
    best_t1 = None
    if t1_keys:
        best_t1 = _safe_float((per_task.get(t1_keys[0]) or {}).get("best_mae"))
    if best_t1 is None or math.isnan(best_t1):
        best_t1, _ = _compute_best_final_from_eval(eval_df, "circuito1")

    _, final_t1_after_last = _compute_best_final_from_eval(eval_df, "circuito1")
    if best_t1 is None or math.isnan(best_t1) \
       or final_t1_after_last is None or math.isnan(final_t1_after_last):
        return out

    forget_abs = max(0.0, final_t1_after_last - best_t1)
    forget_rel = forget_abs / max(1e-9, best_t1)

    out["circuito1_forget_abs"] = forget_abs
    out["circuito1_forget_rel"] = forget_rel
    out["circuito2_forget_abs"] = 0.0
    out["circuito2_forget_rel"] = 0.0
    out["avg_forget_rel"]       = forget_rel
    return out

def _read_continual_results(run_dir: Path) -> dict:
    """Fallback final para MAEs desde continual_results.json."""
    j = _read_json(run_dir / "continual_results.json")
    if not isinstance(j, dict):
        return {}
    c1 = j.get("circuito1", {}) or {}
    c2 = j.get("circuito2", {}) or {}
    return {
        "c1_best":  _safe_float(c1.get("test_mae")),
        "c1_final": _safe_float(c1.get("after_circuito2_mae")),
        "c2_best":  _safe_float(c2.get("test_mae")),
        "c2_final": _safe_float(c2.get("test_mae")),
    }

def build_results_table_from_disk(base_out: Path) -> pd.DataFrame:
    rows = []
    run_dirs = [p for p in base_out.glob("continual_*") if p.is_dir()]
    print(f"[INFO] Escaneando {len(run_dirs)} runs en {base_out}")

    for rd in run_dirs:
        meta      = _parse_basic_meta(rd)
        per_task  = _read_per_task_perf(rd)
        eff_kg, elapsed = _read_efficiency(rd)
        forget_js = _read_forgetting(rd) or {}
        eval_df   = _read_eval_matrix(rd)

        # MAEs por tarea con fallback a eval_matrix y continual_results.json
        c1_best = c1_final = c2_best = c2_final = np.nan

        t1_keys = [k for k in per_task.keys() if "circuito1" in str(k).lower()]
        if t1_keys:
            c1_best  = _safe_float(per_task[t1_keys[0]].get("best_mae"),  default=np.nan)
            c1_final = _safe_float(per_task[t1_keys[0]].get("final_mae"), default=np.nan)
        if math.isnan(c1_best) or math.isnan(c1_final):
            b, f = _compute_best_final_from_eval(eval_df, "circuito1")
            if math.isnan(c1_best):  c1_best  = b
            if math.isnan(c1_final): c1_final = f

        t2_keys = [k for k in per_task.keys() if "circuito2" in str(k).lower()]
        if t2_keys:
            c2_best  = _safe_float(per_task[t2_keys[0]].get("best_mae"),  default=np.nan)
            c2_final = _safe_float(per_task[t2_keys[0]].get("final_mae"), default=np.nan)
        if math.isnan(c2_best) or math.isnan(c2_final):
            b, f = _compute_best_final_from_eval(eval_df, "circuito2")
            if math.isnan(c2_best):  c2_best  = b
            if math.isnan(c2_final): c2_final = f

        # Fallback definitivo: continual_results.json
        if any(math.isnan(x) for x in [c1_best, c1_final, c2_best, c2_final]):
            cr = _read_continual_results(rd)
            if math.isnan(c1_best):  c1_best  = _safe_float(cr.get("c1_best"),  default=c1_best)
            if math.isnan(c1_final): c1_final = _safe_float(cr.get("c1_final"), default=c1_final)
            if math.isnan(c2_best):  c2_best  = _safe_float(cr.get("c2_best"),  default=c2_best)
            if math.isnan(c2_final): c2_final = _safe_float(cr.get("c2_final"), default=c2_final)

        # Olvido (summary/json) o cálculo desde eval_matrix
        f_c1_abs = _safe_float(forget_js.get("circuito1_forget_abs"))
        f_c1_rel = _safe_float(forget_js.get("circuito1_forget_rel"))
        f_c2_abs = _safe_float(forget_js.get("circuito2_forget_abs"))
        f_c2_rel = _safe_float(forget_js.get("circuito2_forget_rel"))
        avg_f_rel = _safe_float(forget_js.get("avg_forget_rel"))

        if all(math.isnan(x) for x in [f_c1_abs, f_c1_rel, f_c2_abs, f_c2_rel, avg_f_rel]):
            comp = _compute_forgetting_from_eval_matrix(eval_df, per_task)
            if comp:
                f_c1_abs = comp.get("circuito1_forget_abs", f_c1_abs)
                f_c1_rel = comp.get("circuito1_forget_rel", f_c1_rel)
                f_c2_abs = comp.get("circuito2_forget_abs", f_c2_abs)
                f_c2_rel = comp.get("circuito2_forget_rel", f_c2_rel)
                avg_f_rel = comp.get("avg_forget_rel", avg_f_rel)

        row = dict(
            run_dir=str(rd.relative_to(base_out)),
            preset=meta["preset"],
            method=meta["method"],
            encoder=meta["encoder"],
            model=meta["model"],
            seed=meta["seed"],
            T=meta["T"],
            batch_size=meta["batch_size"],
            amp=meta["amp"],
            emissions_kg=eff_kg,
            elapsed_sec=elapsed,
            circuito1_best_mae=c1_best,
            circuito1_final_mae=c1_final,
            circuito2_best_mae=c2_best,
            circuito2_final_mae=c2_final,
            circuito1_forget_abs=f_c1_abs,
            circuito1_forget_rel=f_c1_rel,
            circuito2_forget_abs=f_c2_abs,
            circuito2_forget_rel=f_c2_rel,
            avg_forget_rel=avg_f_rel,
        )
        rows.append(row)

    df = pd.DataFrame(rows)

    # Flags extra
    df["is_new_runner"] = df["run_dir"].apply(
        lambda rd: (_abs_run_dir(rd) / "run_row.json").exists()
        or (_abs_run_dir(rd) / "run_row.csv").exists()
    )
    df["mtime"] = df["run_dir"].apply(run_mtime)
    df["mtime_dt"] = pd.to_datetime(df["mtime"], unit="s")
    df["method_base"] = df["method"].astype(str).apply(canonical_method)

    # Tipado numérico (excluimos 'amp', que será boolean)
    numeric_cols = [
        "seed","T","batch_size","emissions_kg","elapsed_sec",
        "circuito1_best_mae","circuito1_final_mae",
        "circuito2_best_mae","circuito2_final_mae",
        "circuito1_forget_abs","circuito1_forget_rel",
        "circuito2_forget_abs","circuito2_forget_rel","avg_forget_rel"
    ]
    for c in numeric_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    # 'amp' como boolean (permite NA)
    if "amp" in df.columns:
        if df["amp"].dtype == bool:
            df["amp"] = df["amp"].astype("boolean")
        else:
            df["amp"] = df["amp"].map(
                lambda v: bool(v) if isinstance(v, (bool,int,float))
                else (str(v).strip().lower() in {"true","1","yes","y"} if isinstance(v, str) else pd.NA)
            ).astype("boolean")

    out_csv = SUMMARY / "results_table_fromdisk.csv"
    df.to_csv(out_csv, index=False)
    print(f"[OK] results_table_fromdisk → {out_csv} | filas:", len(df))
    return df

df_all = build_results_table_from_disk(OUT)
display(df_all.head(3))

nan_cols = [
    "circuito1_best_mae","circuito1_final_mae",
    "circuito2_best_mae","circuito2_final_mae","avg_forget_rel"
]
print("[DEBUG] NaNs globales:", {c:int(df_all[c].isna().sum()) for c in nan_cols if c in df_all.columns})


[INFO] Escaneando 339 runs en /home/cesar/proyectos/TFM_SNN/outputs
[OK] results_table_fromdisk → /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_std_rate_2025-12-11_2042/results_table_fromdisk.csv | filas: 339


,run_dir,preset,method,encoder,model,seed,T,batch_size,amp,emissions_kg,elapsed_sec,circuito1_best_mae,circuito1_final_mae,circuito2_best_mae,circuito2_final_mae,circuito1_forget_abs,circuito1_forget_rel,circuito2_forget_abs,circuito2_forget_rel,avg_forget_rel,is_new_runner,mtime,mtime_dt,method_base
0,continual_fast_as-snn_gr_0.6_lam_0.36_att_f6_f...,fast,as-snn_gr_0.6_lam_0.36_att_f6,rate,PilotNetSNN_66x200_gray,42.0,12.0,320.0,True,NaN,771.873337,0.189499,0.262805,0.190483,0.190483,0.073306,0.386843,0.0,0.0,0.193422,True,1.763134e+09,2025-11-14 15:27:35.800247431,as-snn
1,continual_fast_ewc_lam_5e+07_fast_ewc_l5e7_fb3...,fast,ewc_lam_5e+07,rate,PilotNetSNN_66x200_gray,42.0,12.0,320.0,True,NaN,1921.457078,0.140196,0.231559,0.199714,0.199714,0.091363,0.651682,0.0,0.0,0.325841,True,1.762962e+09,2025-11-12 15:38:42.516785860,ewc
2,continual_fast_as-snn_gr_0.6_lam_0.4_att_f6_fa...,fast,as-snn_gr_0.6_lam_0.4_att_f6,rate,PilotNetSNN_66x200_gray,42.0,12.0,320.0,True,NaN,770.022141,0.127710,0.175707,0.165940,0.165940,0.047996,0.375820,0.0,0.0,0.187910,True,1.763093e+09,2025-11-14 04:09:08.899888992,as-snn


[DEBUG] NaNs globales: {'circuito1_best_mae': 12, 'circuito1_final_mae': 12, 'circuito2_best_mae': 12, 'circuito2_final_mae': 12, 'avg_forget_rel': 12}


<a id="sec-04"></a>
## 4) Selección de runs comparables (`selection_table.csv`)

**Objetivo**

Aplicar filtros “duros” para quedarnos con runs comparables:

- mismos `preset`, `encoder`, `model`, `T`,
- opcionalmente mismo `amp` y `batch_size`,
- runs con ambas tareas evaluadas (sin NaNs en MAEs).

Se genera:

- `selection_table.csv` → tabla de runs filtrados,
- `df_sel` y `df_report` en memoria (para celdas posteriores).

[↑ Volver al índice](#toc)


In [33]:
def _filter_paperset(df: pd.DataFrame, require_both_tasks: bool = True) -> pd.DataFrame:
    m = df.copy()
    if PRESET_KEEP:
        m = m[m["preset"].isin(PRESET_KEEP)]
    if ENCODER_KEEP:
        m = m[m["encoder"].isin(ENCODER_KEEP)]
    if SEED_KEEP:
        m = m[m["seed"].isin(SEED_KEEP)]
    if METHODS_KEEP:
        m = m[m["method_base"].isin(METHODS_KEEP)]
    if ONLY_NEW_RUNNER:
        m = m[m["is_new_runner"] == True]
    if MTIME_FROM is not None:
        m = m[m["mtime_dt"] >= MTIME_FROM]
    if MTIME_TO is not None:
        m = m[m["mtime_dt"] <= MTIME_TO]


    # Runs con ambas tareas evaluadas (mejor y final). Evitamos runs truncados.
    if require_both_tasks:
        must_cols = [
            "circuito1_best_mae","circuito1_final_mae",
            "circuito2_best_mae","circuito2_final_mae"
        ]
        for c in must_cols:
            if c in m.columns:
                m = m[~m[c].isna()]

    print(f"[DEBUG] filtros duros → inicio → {len(df)} runs | post → {len(m)} runs")
    return m

def _comparability_slice(df: pd.DataFrame, strict: bool = True):
    if df.empty:
        return df, {"estrategia":"empty", "kept":0, "dropped":0}
    # Siempre mismo modelo y mismo T
    modes_model = df["model"].dropna().unique().tolist()
    modes_T     = df["T"].dropna().unique().tolist()
    if len(modes_model) > 1:
        top_model = df["model"].value_counts().idxmax()
        df = df[df["model"] == top_model]
    if len(modes_T) > 1:
        top_T = df["T"].value_counts().idxmax()
        df = df[df["T"] == top_T]

    kept_before = len(df)
    if strict:
        # AMP mayoritario (si hay), manteniendo NaN
        if "amp" in df.columns and df["amp"].notna().any():
            top_amp = df["amp"].value_counts(dropna=True).idxmax()
            df = df[(df["amp"].isna()) | (df["amp"] == top_amp)]
        # batch_size mayoritario (si hay), manteniendo NaN
        if "batch_size" in df.columns and df["batch_size"].notna().any():
            top_bs = df["batch_size"].value_counts(dropna=True).idxmax()
            df = df[(df["batch_size"].isna()) | (df["batch_size"] == top_bs)]
        strategy = "strict:model+T(+amp+batch)"
    else:
        strategy = "relaxed:model+T"

    kept_after = len(df)
    return df, {"estrategia": strategy, "kept": kept_after, "dropped": kept_before - kept_after}

df_sel0 = _filter_paperset(df_all, require_both_tasks=True)
df_sel, stats = _comparability_slice(df_sel0, strict=STRICT_CFG)
if df_sel.empty and STRICT_CFG:
    print("[WARN] Comparabilidad dejó 0 runs. Relajando AMP y batch_size…")
    df_sel, stats = _comparability_slice(df_sel0, strict=False)

# Copia para informes (se puede filtrar naive aquí si se desea solo de cara a tablas/plots)
df_sel["batch_size_filled"] = df_sel["batch_size"].copy()
df_report = df_sel.copy()
if IGNORE_NAIVE_IN_REPORTS and "method_base" in df_report.columns:
    df_report = df_report[df_report["method_base"] != "naive"]

display(df_sel.head(10))
print(f"[OK] Selección final: {len(df_sel)} runs | estrategia: {stats}")

out_csv = SUMMARY / "selection_table.csv"
df_sel.to_csv(out_csv, index=False)
print("[OK] Selección →", out_csv)

# Guardamos ruta en _last.json
_last_update(selection=out_csv)

[DEBUG] filtros duros → inicio → 339 runs | post → 62 runs


,run_dir,preset,method,encoder,model,seed,T,batch_size,amp,emissions_kg,elapsed_sec,circuito1_best_mae,circuito1_final_mae,circuito2_best_mae,circuito2_final_mae,circuito1_forget_abs,circuito1_forget_rel,circuito2_forget_abs,circuito2_forget_rel,avg_forget_rel,is_new_runner,mtime,mtime_dt,method_base,batch_size_filled
6,continual_std_sca-snn_bins50_beta0.45_bias-0.0...,std,sca-snn_bins50_beta0.45_bias-0.05_temp0.45_ab2...,rate,PilotNetSNN_66x200_gray,42.0,18.0,256.0,True,0.061494,9124.745457,0.117477,0.156539,0.160436,0.160436,0.039062,0.332509,0.0,0.0,0.166255,True,1.763456e+09,2025-11-18 08:54:46.666387081,sca-snn,256.0
10,continual_std_as-snn_gr_0.5_lam_0.18_ema_0.9_l...,std,as-snn_gr_0.5_lam_0.18_ema_0.9_l1_scale_on,rate,PilotNetSNN_66x200_gray,42.0,18.0,256.0,True,0.097530,14240.604300,0.117562,0.147439,0.154159,0.154159,0.029877,0.254135,0.0,0.0,0.127067,True,1.763815e+09,2025-11-22 12:41:10.685746908,as-snn,256.0
11,continual_std_sca-snn_bins64_beta0.52_bias0_te...,std,sca-snn_bins64_beta0.52_bias0_temp0.45_ab32_flat0,rate,PilotNetSNN_66x200_gray,42.0,18.0,256.0,True,0.083097,14572.243987,0.121406,0.185356,0.160054,0.160054,0.063950,0.526744,0.0,0.0,0.263372,True,1.763540e+09,2025-11-19 08:18:57.654322386,sca-snn,256.0
14,continual_std_sa-snn_top_sa_k012_tau32_vt120_s...,std,sa-snn,rate,PilotNetSNN_66x200_gray,1007.0,18.0,256.0,True,0.099314,14559.295749,0.118604,0.157633,0.153774,0.153774,0.039029,0.329066,0.0,0.0,0.164533,True,1.763777e+09,2025-11-22 02:01:12.537743092,sa-snn,256.0
17,continual_std_sa-snn_tune_sa_k013_vt120_rate_m...,std,sa-snn,rate,PilotNetSNN_66x200_gray,42.0,18.0,256.0,True,0.063431,9355.478259,0.118192,0.144487,0.154740,0.154740,0.026295,0.222474,0.0,0.0,0.111237,True,1.763957e+09,2025-11-24 04:04:40.539897919,sa-snn,256.0
21,continual_std_as-snn_gr_0.5_lam_0.16_ema_0.9_l...,std,as-snn_gr_0.5_lam_0.16_ema_0.9_l1_scale_on,rate,PilotNetSNN_66x200_gray,42.0,18.0,256.0,True,0.084100,14852.069115,0.115306,0.163676,0.153717,0.153717,0.048369,0.419485,0.0,0.0,0.209743,True,1.763792e+09,2025-11-22 06:08:54.860842705,as-snn,256.0
27,continual_std_as-snn_gr_0.5_lam_0.2_ema_0.9_l1...,std,as-snn_gr_0.5_lam_0.2_ema_0.9_l1_scale_on,rate,PilotNetSNN_66x200_gray,87.0,18.0,256.0,True,0.096461,14135.350830,0.119589,0.156567,0.154498,0.154498,0.036978,0.309210,0.0,0.0,0.154605,True,1.763649e+09,2025-11-20 14:33:14.859453440,as-snn,256.0
29,continual_std_sca-snn_bins50_beta0.48_bias0_te...,std,sca-snn_bins50_beta0.48_bias0_temp0.5_ab24_flat1,rate,PilotNetSNN_66x200_gray,42.0,18.0,256.0,True,0.063443,9343.780607,0.121992,0.166626,0.160129,0.160129,0.044634,0.365876,0.0,0.0,0.182938,True,1.763872e+09,2025-11-23 04:34:19.184135199,sca-snn,256.0
37,continual_std_ewc_lam_3e+08_std_ewc_l3e8_fb512...,std,ewc_lam_3e+08,rate,PilotNetSNN_66x200_gray,42.0,18.0,256.0,True,0.063458,9278.637108,0.121088,0.247564,0.212777,0.212777,0.126477,1.044505,0.0,0.0,0.522253,True,1.763355e+09,2025-11-17 04:54:29.336913586,ewc,256.0
67,continual_std_as-snn_gr_0.55_lam_0.25_ema_0.9_...,std,as-snn_gr_0.55_lam_0.25_ema_0.9_l1_scale_on,rate,PilotNetSNN_66x200_gray,42.0,18.0,256.0,True,0.061392,8946.973210,0.119720,0.158855,0.157833,0.157833,0.039135,0.326884,0.0,0.0,0.163442,True,1.763391e+09,2025-11-17 14:51:28.554344893,as-snn,256.0


[OK] Selección final: 62 runs | estrategia: {'estrategia': 'strict:model+T(+amp+batch)', 'kept': 62, 'dropped': 0}
[OK] Selección → /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_std_rate_2025-12-11_2042/selection_table.csv
[LAST] actualizado → /home/cesar/proyectos/TFM_SNN/outputs/summary/_last.json


<a id="sec-05"></a>
## 5) Ranking compuesto y *winners* por método

**Objetivo**

- Definir un **score compuesto** entre:
  - `circuito2_final_mae` (rendimiento final),
  - `avg_forget_rel` (olvido medio).
- Escoger un *winner* por método (base o completo) según ese score.
- Exportar a `winners_per_methodbase.csv` o `winners_per_fullmethod.csv`.

[↑ Volver al índice](#toc)


In [34]:
def _ensure_mae_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Crea alias entre c2_final_mae y circuito2_final_mae si falta alguno."""
    df = df.copy()
    has_c2    = "c2_final_mae" in df.columns
    has_circ2 = "circuito2_final_mae" in df.columns
    if has_c2 and not has_circ2:
        df["circuito2_final_mae"] = df["c2_final_mae"]
    elif has_circ2 and not has_c2:
        df["c2_final_mae"] = df["circuito2_final_mae"]
    return df

def _minmax_norm(s: pd.Series) -> pd.Series:
    s = pd.to_numeric(s, errors="coerce")
    if s.size == 0:
        return s
    vals = s.values.astype(float)
    finite = np.isfinite(vals)
    if finite.sum() == 0:
        return pd.Series(np.full(len(s), np.nan), index=s.index)
    lo, hi = float(np.nanmin(vals)), float(np.nanmax(vals))
    if not np.isfinite(lo) or not np.isfinite(hi) or hi <= lo:
        out = np.zeros(len(s), dtype=float)
        out[~finite] = np.nan
        return pd.Series(out, index=s.index)
    return (s - lo) / (hi - lo)

# 1) Copia y compatibilidad de columnas
df_rank = _ensure_mae_columns(df_report)

# Columnas clave
MAE_COL  = "circuito2_final_mae"
FORG_COL = "avg_forget_rel"

# 2) Si está vacío, escribe CSV vacío y sal con aviso
if df_rank.empty:
    winners = df_rank.copy()
    out_csv = SUMMARY / "winners_per_methodbase.csv"
    winners.to_csv(out_csv, index=False)
    print("[WARN] df_rank está vacío tras filtros. Winners vacío →", out_csv)
    display(winners)  # DF vacío
else:
    mae_norm  = _minmax_norm(df_rank[MAE_COL]  if MAE_COL  in df_rank.columns else pd.Series([], dtype=float))
    forg_norm = _minmax_norm(df_rank[FORG_COL] if FORG_COL in df_rank.columns else pd.Series([], dtype=float))

    score_comp = ALPHA_COMPOSITE * mae_norm + (1 - ALPHA_COMPOSITE) * forg_norm
    score_eff  = score_comp.copy()
    if len(score_eff) != 0:
        score_eff[forg_norm.isna()] = mae_norm[forg_norm.isna()]  # fallback

    df_rank["score_comp"] = score_comp
    df_rank["score_eff"]  = score_eff

    # Agrupación: por método base o por método completo (separa composites)
    group_col = "method" if GROUP_BY_FULL_METHOD else "method_base"
    if group_col not in df_rank.columns:
        df_rank[group_col] = df_rank["method"] if "method" in df_rank.columns else "unknown"

    # Winners
    if df_rank["score_eff"].notna().any():
        winners = (
            df_rank
            .sort_values([group_col, "score_eff", MAE_COL], ascending=[True, True, True])
            .groupby(group_col, as_index=False)
            .head(1)
            .sort_values("score_eff", ascending=True)
        )
    else:
        winners = (
            df_rank
            .sort_values([group_col, MAE_COL], ascending=[True, True])
            .groupby(group_col, as_index=False)
            .head(1)
        )

    # Export
    out_csv = SUMMARY / f"winners_per_{'fullmethod' if GROUP_BY_FULL_METHOD else 'methodbase'}.csv"
    winners.to_csv(out_csv, index=False)

    n_rows = len(df_rank)
    n_groups = df_rank[group_col].nunique()
    print(f"[OK] Winners → {out_csv} | candidatos={n_rows} | grupos={n_groups}")
    cols_show = [c for c in ["run_dir", group_col, "seed", MAE_COL, FORG_COL, "emissions_kg", "score_eff"]
                 if c in winners.columns]
    display(winners[cols_show] if cols_show else winners)

# Guardamos ruta en _last.json
_last_update(winners=out_csv)

[OK] Winners → /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_std_rate_2025-12-11_2042/winners_per_methodbase.csv | candidatos=62 | grupos=5


,run_dir,method_base,seed,circuito2_final_mae,avg_forget_rel,emissions_kg,score_eff
140,continual_std_sca-snn_bins50_beta0.5_bias0_tem...,sca-snn,42.0,0.154640,0.093250,0.051350,0.055552
286,continual_std_as-snn_gr_0.5_lam_0.2_ema_0.9_l1...,as-snn,42.0,0.150619,0.132559,0.061587,0.068938
187,continual_std_rehearsal_buf_3000_rr_30_std_reh...,rehearsal,777.0,0.159508,0.074368,0.062230,0.071395
17,continual_std_sa-snn_tune_sa_k013_vt120_rate_m...,sa-snn,42.0,0.154740,0.111237,0.063431,0.076388
100,continual_std_ewc_lam_3e+06_std_ewc_l3e6_fb200...,ewc,42.0,0.170263,0.234944,0.062440,0.332237


[LAST] actualizado → /home/cesar/proyectos/TFM_SNN/outputs/summary/_last.json


<a id="sec-06"></a>
## 6) Comparativa relativa frente a EWC

**Objetivo**

- Construir una tabla con métricas **relativas** frente a un baseline (EWC por defecto):
  - ΔMAE vs baseline (`delta_mae_vs_base`),
  - Δolvido vs baseline (`delta_forget_vs_base`),
  - porcentaje de mejora en MAE (`mae_improv_pct`),
  - cambio en olvido en puntos porcentuales (`forget_delta_pp`).
- Generar una tabla tipo *scorecard* por método:
  - `scorecards_por_metodo_vs_ewc.csv`.

> Esto es útil para escribir frases del estilo:
> “SA-SNN mejora un X% el MAE en Circuito 2 respecto a EWC...”.

[↑ Volver al índice](#toc)


In [35]:
def _row_key_for_match(r):
    """Clave de emparejamiento para baseline estricto."""
    return (
        r.get("preset"), r.get("encoder"), r.get("model"), r.get("T"),
        r.get("amp"), r.get("batch_size"), r.get("seed")
    )

def compute_relative_to_baseline(df_in: pd.DataFrame, baseline="naive", strict=True):
    if not baseline:
        return None
    if isinstance(baseline, str) and baseline.lower() == "auto":
        bases = df_in["method_base"].dropna().str.lower().unique().tolist()
        baseline = "ewc" if "ewc" in bases else ("naive" if "naive" in bases else None)
        if not baseline:
            return None

    dfb = df_in.copy()
    dfb["__key__"] = dfb.apply(
        lambda r: (
            r.get("preset"), r.get("encoder"), r.get("model"), r.get("T"),
            r.get("amp"), r.get("batch_size"), r.get("seed")
        ),
        axis=1
    )

    # Para cada clave, escoge el run baseline con menor MAE
    base_rows = []
    mask_base = dfb["method_base"].astype(str).str.lower() == baseline.lower()
    for key, g in dfb[mask_base].groupby("__key__"):
        if g.empty:
            continue
        idx = g["circuito2_final_mae"].astype(float).idxmin()
        row = g.loc[idx]
        base_rows.append({
            "__key__": key,
            "baseline_mae": float(row["circuito2_final_mae"]),
            "baseline_forget": float(row["avg_forget_rel"])
        })
    base_tbl = pd.DataFrame(base_rows)

    out = dfb.merge(base_tbl, on="__key__", how="left")
    out["delta_mae_vs_base"]    = out["circuito2_final_mae"] - out["baseline_mae"]
    out["delta_forget_vs_base"] = out["avg_forget_rel"]      - out["baseline_forget"]
    return out

if RELATIVE_BASELINE:
    df_rel = compute_relative_to_baseline(df_sel, baseline=RELATIVE_BASELINE, strict=BASELINE_MATCH_STRICT)
    if df_rel is not None:
        out_csv = SUMMARY / f"relative_to_{RELATIVE_BASELINE}.csv"
        df_rel.to_csv(out_csv, index=False)
        print(f"[OK] Relative-to-baseline → {out_csv}")

        _last_update(relative_baseline=RELATIVE_BASELINE, relative_csv=out_csv)

        # Top-10 mejoras vs baseline (orden: delta_mae luego delta_forget)
        mask_has_base = (~df_rel["baseline_mae"].isna()) & (~df_rel["baseline_forget"].isna())
        top_improve = (
            df_rel[mask_has_base]
            .sort_values(["delta_mae_vs_base","delta_forget_vs_base"], ascending=[True, True])
            .head(10)
        )
        cols_show = [
            "run_dir","method","method_base","circuito2_final_mae","avg_forget_rel",
            "baseline_mae","baseline_forget","delta_mae_vs_base","delta_forget_vs_base"
        ]
        display(top_improve[cols_show])

        # Scorecards por método vs EWC
        rel = df_rel.copy()
        rel["mae_improv_pct"] = 100.0 * (rel["baseline_mae"] - rel["circuito2_final_mae"]) / rel["baseline_mae"]
        rel["forget_delta_pp"] = 100.0 * (rel["avg_forget_rel"] - rel["baseline_forget"])

        key_cols = [
            "run_dir","method","method_base","preset","encoder","model","seed","T","batch_size","amp"
        ]
        cols_metrics = [
            "circuito2_final_mae","avg_forget_rel","baseline_mae","baseline_forget",
            "mae_improv_pct","forget_delta_pp","emissions_kg","elapsed_sec"
        ]

        # Aseguramos winners (si no quedó en memoria)
        try:
            winners
        except NameError:
            tmp = df_rel.copy()
            if "c2_final_mae" not in tmp.columns and "circuito2_final_mae" in tmp.columns:
                tmp["c2_final_mae"] = tmp["circuito2_final_mae"]
            if "score_eff" not in tmp.columns:
                tmp["score_eff"] = np.nan
            group_col = "method_base" if "method_base" in tmp.columns else "method"
            winners = (
                tmp.sort_values([group_col, "c2_final_mae"], ascending=[True, True])
                .groupby(group_col, as_index=False)
                .head(1)
            )

        scorecards = winners[key_cols].merge(rel[key_cols + cols_metrics], on=key_cols, how="left")

        out_dir = SUMMARY
        scorecards_csv = out_dir / "scorecards_por_metodo_vs_ewc.csv"
        scorecards.to_csv(scorecards_csv, index=False)
        print("[OK] Scorecards →", scorecards_csv)

        # (Opcional) convertir MAE a grados si las etiquetas son [-1,1]
        STEER_REP = os.environ.get("STEER_REP", "unknown")  # "norm1" si [-1,1], "deg" si ya son grados
        DEG_RANGE = float(os.environ.get("DEG_RANGE", "25"))

        def _mae_to_deg(mae):
            try:
                v = float(mae)
                if STEER_REP.lower() == "norm1":
                    return v * DEG_RANGE
                return v
            except Exception:
                return np.nan

        scorecards["c2_final_mae_deg"] = scorecards["circuito2_final_mae"].apply(_mae_to_deg)
        scorecards.to_csv(scorecards_csv, index=False)  # re-escribe con la columna extra
        print("[OK] Añadido c2_final_mae_deg (si procede) →", scorecards_csv)

        show_cols = [
            "method_base","method","circuito2_final_mae","c2_final_mae_deg","avg_forget_rel",
            "baseline_mae","baseline_forget","mae_improv_pct","forget_delta_pp","emissions_kg"
        ]
        disp = (
            scorecards.sort_values(["mae_improv_pct","forget_delta_pp"], ascending=[False, True])
        )[show_cols]
        display(disp)

[OK] Relative-to-baseline → /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_std_rate_2025-12-11_2042/relative_to_ewc.csv
[LAST] actualizado → /home/cesar/proyectos/TFM_SNN/outputs/summary/_last.json


,run_dir,method,method_base,circuito2_final_mae,avg_forget_rel,baseline_mae,baseline_forget,delta_mae_vs_base,delta_forget_vs_base
43,continual_std_as-snn_gr_0.5_lam_0.22_ema_0.92_...,as-snn_gr_0.5_lam_0.22_ema_0.92_l1_scale_on,as-snn,0.150500,0.187298,0.170263,0.234944,-0.019763,-0.047646
53,continual_std_as-snn_gr_0.5_lam_0.2_ema_0.9_l1...,as-snn_gr_0.5_lam_0.2_ema_0.9_l1_scale_on,as-snn,0.150619,0.132559,0.170263,0.234944,-0.019644,-0.102386
51,continual_std_as-snn_gr_0.5_lam_0.2_ema_0.9_l1...,as-snn_gr_0.5_lam_0.2_ema_0.9_l1_scale_on,as-snn,0.152216,0.146668,0.170263,0.234944,-0.018048,-0.088277
5,continual_std_as-snn_gr_0.5_lam_0.16_ema_0.9_l...,as-snn_gr_0.5_lam_0.16_ema_0.9_l1_scale_on,as-snn,0.153717,0.209743,0.170263,0.234944,-0.016546,-0.025202
40,continual_std_as-snn_gr_0.5_lam_0.22_ema_0.9_l...,as-snn_gr_0.5_lam_0.22_ema_0.9_l1_scale_on,as-snn,0.153929,0.217123,0.170263,0.234944,-0.016334,-0.017821
58,continual_std_sca-snn_bins50_beta0.52_bias0_te...,sca-snn_bins50_beta0.52_bias0_temp0.5_ab24_flat1,sca-snn,0.153973,0.200748,0.170263,0.234944,-0.016290,-0.034196
1,continual_std_as-snn_gr_0.5_lam_0.18_ema_0.9_l...,as-snn_gr_0.5_lam_0.18_ema_0.9_l1_scale_on,as-snn,0.154159,0.127067,0.170263,0.234944,-0.016104,-0.107877
46,continual_std_sca-snn_bins50_beta0.45_bias-0.0...,sca-snn_bins50_beta0.45_bias-0.05_temp0.45_ab2...,sca-snn,0.154628,0.205157,0.170263,0.234944,-0.015635,-0.029788
25,continual_std_sca-snn_bins50_beta0.5_bias0_tem...,sca-snn_bins50_beta0.5_bias0_temp0.5_ab24_flat0,sca-snn,0.154640,0.093250,0.170263,0.234944,-0.015623,-0.141695
21,continual_std_sca-snn_bins50_beta0.5_bias0_tem...,sca-snn_bins50_beta0.5_bias0_temp0.5_ab24_flat1,sca-snn,0.154702,0.160906,0.170263,0.234944,-0.015562,-0.074038


[OK] Scorecards → /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_std_rate_2025-12-11_2042/scorecards_por_metodo_vs_ewc.csv
[OK] Añadido c2_final_mae_deg (si procede) → /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_std_rate_2025-12-11_2042/scorecards_por_metodo_vs_ewc.csv


,method_base,method,circuito2_final_mae,c2_final_mae_deg,avg_forget_rel,baseline_mae,baseline_forget,mae_improv_pct,forget_delta_pp,emissions_kg
1,as-snn,as-snn_gr_0.5_lam_0.2_ema_0.9_l1_scale_on,0.150619,0.150619,0.132559,0.170263,0.234944,11.537431,-10.238568,0.061587
0,sca-snn,sca-snn_bins50_beta0.5_bias0_temp0.5_ab24_flat0,0.154640,0.154640,0.093250,0.170263,0.234944,9.175961,-14.169467,0.051350
3,sa-snn,sa-snn,0.154740,0.154740,0.111237,0.170263,0.234944,9.117486,-12.370741,0.063431
4,ewc,ewc_lam_3e+06,0.170263,0.170263,0.234944,0.170263,0.234944,0.000000,0.000000,0.062440
2,rehearsal,rehearsal_buf_3000_rr_30,0.159508,0.159508,0.074368,NaN,NaN,NaN,NaN,0.062230


<a id="sec-07"></a>
## 7) Top-6 global + frente de Pareto y *scatter* MAE vs olvido

**Objetivo**

- Seleccionar el **Top-6** global según el score compuesto (`top6_composite.csv`).
- Calcular un **frente de Pareto** minimizando MAE y olvido (`pareto.csv`).
- Generar *scatters* sencillos MAE vs olvido:
  - todos los runs (`scatter_all.png`),
  - solo winners (`scatter_winners.png`).

[↑ Volver al índice](#toc)

In [36]:
import matplotlib.pyplot as plt

# Helpers por si no vienes de la celda 5 (definidos ya arriba)
try:
    _ensure_mae_columns
except NameError:
    def _ensure_mae_columns(df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        has_c2    = "c2_final_mae" in df.columns
        has_circ2 = "circuito2_final_mae" in df.columns
        if has_c2 and not has_circ2:
            df["circuito2_final_mae"] = df["c2_final_mae"]
        elif has_circ2 and not has_c2:
            df["c2_final_mae"] = df["circuito2_final_mae"]
        return df

df_rank2 = _ensure_mae_columns(df_report if "df_report" in globals() else df_sel.copy())
if "score_eff" not in df_rank2.columns:
    df_rank2 = df_rank2.copy()
    df_rank2["score_eff"] = np.nan
if "method_base" not in df_rank2.columns:
    df_rank2["method_base"] = df_rank2["method"] if "method" in df_rank2.columns else "unknown"

MAE_COL = "circuito2_final_mae" if "circuito2_final_mae" in df_rank2.columns \
          else ("c2_final_mae" if "c2_final_mae" in df_rank2.columns else "circuito2_final_mae")

# Top-N compuesto
TOPN = 6
if df_rank2.empty:
    topn = df_rank2.copy()
    print("[WARN] df_rank está vacío; Top-N vacío.")
else:
    cols_sort = [c for c in ["score_eff", MAE_COL] if c in df_rank2.columns]
    if not cols_sort:
        topn = df_rank2.head(TOPN)
    else:
        topn = df_rank2.sort_values(cols_sort, ascending=[True] * len(cols_sort)).head(TOPN)

out_csv = SUMMARY / "top6_composite.csv"
topn.to_csv(out_csv, index=False)
print("[OK] Top-6 →", out_csv)
_last_update(top6=out_csv)

cols_show = [
    c for c in ["run_dir","method_base","seed",MAE_COL,"avg_forget_rel","emissions_kg","score_eff"]
    if c in topn.columns
]
display(topn[cols_show] if cols_show else topn)

# Pareto (minimiza MAE y olvido); NaN -> +inf para no dominar
tmp = _ensure_mae_columns(df_report.copy())
if "method_base" not in tmp.columns:
    tmp["method_base"] = tmp["method"] if "method" in tmp.columns else "unknown"
if MAE_COL not in tmp.columns:
    tmp[MAE_COL] = np.nan

tmp["_olvido_for_pareto"] = pd.to_numeric(tmp.get("avg_forget_rel", np.nan), errors="coerce")
tmp.loc[tmp["_olvido_for_pareto"].isna(), "_olvido_for_pareto"] = np.inf
tmp["_mae_for_pareto"] = pd.to_numeric(tmp.get(MAE_COL, np.nan), errors="coerce")
tmp.loc[tmp["_mae_for_pareto"].isna(), "_mae_for_pareto"] = np.inf

pareto_idx = []
vals = tmp[["_mae_for_pareto", "_olvido_for_pareto"]].values
for i in range(len(tmp)):
    mae_i, forg_i = vals[i]
    dominated = False
    for j in range(len(tmp)):
        if j == i: 
            continue
        mae_j, forg_j = vals[j]
        if (mae_j <= mae_i) and (forg_j <= forg_i) and ((mae_j < mae_i) or (forg_j < forg_i)):
            dominated = True
            break
    if not dominated:
        pareto_idx.append(i)

df_pareto = tmp.iloc[pareto_idx].drop(columns=["_olvido_for_pareto", "_mae_for_pareto"])
out_csv = SUMMARY / "pareto.csv"
df_pareto.to_csv(out_csv, index=False)
print("[OK] Pareto →", out_csv)
_last_update(pareto=out_csv)

cols_show = [c for c in ["run_dir","method_base",MAE_COL,"avg_forget_rel"] if c in df_pareto.columns]
display(df_pareto[cols_show] if cols_show else df_pareto)

# Scatters MAE vs olvido
def _scatter(df, title, out_png: Path):
    if df.empty or "avg_forget_rel" not in df.columns or "circuito2_final_mae" not in df.columns:
        print(f"[WARN] No se puede dibujar scatter para {title} (faltan columnas o df vacío)")
        return
    x = pd.to_numeric(df["avg_forget_rel"], errors="coerce")
    y = pd.to_numeric(df["circuito2_final_mae"], errors="coerce")
    plt.figure(figsize=(7,5))
    plt.scatter(x, y)  # sin estilos ni colores específicos
    plt.xlabel("avg_forget_rel (↓ mejor)")
    plt.ylabel("circuito2_final_mae (↓ mejor)")
    plt.title(title)
    plt.grid(True, which="both", linestyle="--", alpha=0.3)
    plt.tight_layout()
    plt.savefig(out_png, dpi=160)
    plt.close()

_scatter(df_sel, "Todos", SUMMARY / "scatter_all.png")
_scatter(winners if "winners" in globals() else df_sel.head(0),
         "Winners por método", SUMMARY / "scatter_winners.png")
print("[OK] Scatter →", SUMMARY / "scatter_all.png")
print("[OK] Scatter →", SUMMARY / "scatter_winners.png")

_last_update(scatter_all=SUMMARY / "scatter_all.png",
             scatter_winners=SUMMARY / "scatter_winners.png")


[OK] Top-6 → /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_std_rate_2025-12-11_2042/top6_composite.csv
[LAST] actualizado → /home/cesar/proyectos/TFM_SNN/outputs/summary/_last.json


,run_dir,method_base,seed,circuito2_final_mae,avg_forget_rel,emissions_kg,score_eff
83,continual_std_sca-snn_bins50_beta0.5_bias0_tem...,sca-snn,1007.0,0.150095,0.147925,0.075241,NaN
249,continual_std_as-snn_gr_0.5_lam_0.22_ema_0.92_...,as-snn,42.0,0.150500,0.187298,0.071094,NaN
286,continual_std_as-snn_gr_0.5_lam_0.2_ema_0.9_l1...,as-snn,42.0,0.150619,0.132559,0.061587,NaN
277,continual_std_as-snn_gr_0.5_lam_0.2_ema_0.9_l1...,as-snn,42.0,0.152216,0.146668,0.100206,NaN
21,continual_std_as-snn_gr_0.5_lam_0.16_ema_0.9_l...,as-snn,42.0,0.153717,0.209743,0.084100,NaN
14,continual_std_sa-snn_top_sa_k012_tau32_vt120_s...,sa-snn,1007.0,0.153774,0.164533,0.099314,NaN


[OK] Pareto → /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_std_rate_2025-12-11_2042/pareto.csv
[LAST] actualizado → /home/cesar/proyectos/TFM_SNN/outputs/summary/_last.json


,run_dir,method_base,circuito2_final_mae,avg_forget_rel
10,continual_std_as-snn_gr_0.5_lam_0.18_ema_0.9_l...,as-snn,0.154159,0.127067
83,continual_std_sca-snn_bins50_beta0.5_bias0_tem...,sca-snn,0.150095,0.147925
140,continual_std_sca-snn_bins50_beta0.5_bias0_tem...,sca-snn,0.154640,0.093250
187,continual_std_rehearsal_buf_3000_rr_30_std_reh...,rehearsal,0.159508,0.074368
286,continual_std_as-snn_gr_0.5_lam_0.2_ema_0.9_l1...,as-snn,0.150619,0.132559


[OK] Scatter → /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_std_rate_2025-12-11_2042/scatter_all.png
[OK] Scatter → /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_std_rate_2025-12-11_2042/scatter_winners.png
[LAST] actualizado → /home/cesar/proyectos/TFM_SNN/outputs/summary/_last.json


<a id="sec-08"></a>
## 8) Top-5 por método (tabla + LaTeX)

**Objetivo**

- Para cada `method_base`, seleccionar sus **5 mejores configuraciones** (según MAE/olvido/emisiones).
- Parsear hiperparámetros a partir del nombre del método (gr, lam, k, tau, vt, p, bins, beta, bias, temp, etc.).
- Exportar:
  - `top5_por_metodo.csv` (tabla legible),
  - `top5_por_metodo.tex` (tablas LaTeX por método).

> Esto te va a servir para construir tablas detalladas en la memoria “por método”.

[↑ Volver al índice](#toc)


In [37]:
# Asegura que tenemos df de selección
try:
    OUT
except NameError:
    OUT = Path(os.environ.get("OUT_DIR", "/home/cesar/proyectos/TFM_SNN/outputs")).resolve()

try:
    _last_update
except NameError:
    def _last_update(**kwargs):
        pass  # no-op si no está definida

LAST_PATH = OUT / "summary" / "_last.json"
LATEST = OUT / "summary" / "latest"

def _pick_latest_selection():
    # 1) _last.json
    try:
        last = json.loads(LAST_PATH.read_text(encoding="utf-8"))
        p = Path(last.get("selection", ""))
        if p.exists():
            return p
    except Exception:
        pass
    # 2) SUMMARY actual
    try:
        if "SUMMARY" in globals():
            p = Path(SUMMARY) / "selection_table.csv"
            if p.exists():
                return p
    except Exception:
        pass
    # 3) symlink latest
    try:
        p = LATEST / "selection_table.csv"
        if p.exists():
            return p
    except Exception:
        pass
    # 4) glob: el más reciente
    cands = list(OUT.glob("summary/*/selection_table.csv"))
    if not cands:
        raise FileNotFoundError("No se encontró ningún selection_table.csv en outputs/summary/**/")
    cands.sort(key=lambda x: x.stat().st_mtime, reverse=True)
    return cands[0]

# 1) Carga selección
sel_csv = _pick_latest_selection()
df = pd.read_csv(sel_csv)
print(f"[LOAD] selection_table.csv → {sel_csv}")

# 2) Normaliza columnas clave
def _canonical_method2(m: str) -> str:
    s = (m or "").lower()
    if s.startswith("ewc"):
        return "ewc"
    if s.startswith("as-snn"):
        return "as-snn"
    if s.startswith("sa-snn"):
        return "sa-snn"
    if s.startswith("sca-snn"):
        return "sca-snn"
    if s.startswith("rehearsal"):
        return "rehearsal"
    if s.startswith("naive"):
        return "naive"
    return s or "unknown"

if "method_base" not in df.columns or df["method_base"].isna().all():
    df["method_base"] = df["method"].astype(str).apply(_canonical_method2)
else:
    mask_na = df["method_base"].isna()
    if mask_na.any():
        df.loc[mask_na, "method_base"] = df.loc[mask_na, "method"].astype(str).apply(_canonical_method2)

# 3bis) Hiperparámetros desde method_params.json
def _mp_for_run(run_dir_str: str) -> dict:
    rd = _abs_run_dir(run_dir_str)
    return _read_method_params(rd)

params_json = df["run_dir"].apply(_mp_for_run).apply(pd.Series)
# Prefijamos para no colisionar con columnas existentes
params_json = params_json.add_prefix("mp_")

# Combinamos: df base + JSON de params
dfp = pd.concat([df, params_json], axis=1)

# 4) Métrica objetivo y orden
for col_src, col_dst in [
    ("circuito2_final_mae","c2_final_mae"),
    ("avg_forget_rel","forget"),
    ("emissions_kg","co2")
]:
    if col_src in dfp.columns:
        dfp[col_dst] = pd.to_numeric(dfp[col_src], errors="coerce")
    else:
        dfp[col_dst] = np.nan

# --- NUEVO: guardamos selección enriquecida con mp_* para usarla en 06_ANALISIS_TFM ---
export_dir = sel_csv.parent
sel_with_params_csv = export_dir / "selection_with_params.csv"
dfp.to_csv(sel_with_params_csv, index=False)
print(f"[OK] selection_with_params.csv → {sel_with_params_csv}")

# 5) Top-5 por method_base (a nivel de run, lo mantenemos para exploración puntual)
tops = []
for mb, g in dfp.groupby("method_base", dropna=False):
    g = g.sort_values(["c2_final_mae","forget","co2"], ascending=[True, True, True]).head(5)
    # columnas procedentes de method_params.json (mp_*)
    mp_cols = [c for c in g.columns if c.startswith("mp_")]
    keep_cols = ["method_base","method","c2_final_mae","forget","co2"] + sorted(mp_cols)
    tops.append(g[keep_cols])

if tops:
    top5 = pd.concat(tops, ignore_index=True)
else:
    top5 = dfp.head(0)

# 6) Exporta en el SUMMARY correspondiente a la selección
out_csv = export_dir / "top5_por_metodo.csv"
out_tex = export_dir / "top5_por_metodo.tex"

top5.to_csv(out_csv, index=False)
with open(out_tex, "w", encoding="utf-8") as f:
    for mb, g in top5.groupby("method_base", dropna=False):
        base_cols = ["method","c2_final_mae","forget","co2"]
        other_cols = [c for c in g.columns if c not in (["method_base"] + base_cols)]
        gg = g[base_cols + other_cols]
        f.write(gg.to_latex(index=False, float_format="%.6f"))

print(f"[OK] → {out_csv.name} / {out_tex.name} en {export_dir}")
_last_update(top5_csv=out_csv, top5_tex=out_tex)

# (Opcional) mostrar por pantalla
print("\n[Vista rápida Top-5 por método]")
for mb, g in top5.groupby("method_base", dropna=False):
    print(f"\n### {mb} — {len(g)} configs")
    display(g)


[LOAD] selection_table.csv → /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_std_rate_2025-12-11_2042/selection_table.csv
[OK] selection_with_params.csv → /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_std_rate_2025-12-11_2042/selection_with_params.csv
[OK] → top5_por_metodo.csv / top5_por_metodo.tex en /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_std_rate_2025-12-11_2042
[LAST] actualizado → /home/cesar/proyectos/TFM_SNN/outputs/summary/_last.json

[Vista rápida Top-5 por método]

### as-snn — 5 configs


,method_base,method,c2_final_mae,forget,co2,mp_activity_every,mp_activity_verbose,mp_anchor_batches,mp_assume_binary_spikes,mp_attach_to,mp_beta,mp_bias,mp_buffer_size,mp_do_synaptic_scaling,mp_ema,mp_fisher_batches,mp_flatten_spatial,mp_gamma_ratio,mp_habit_decay,mp_k,mp_lambd,mp_lambda_a,mp_log_every,mp_measure_at,mp_num_bins,mp_p,mp_penalty_mode,mp_replay_ratio,mp_reset_counters_each_task,mp_soft_mask_temp,mp_target_active_frac,mp_tau,mp_th_max,mp_th_min,mp_update_on_eval,mp_verbose,mp_vt_scale
0,as-snn,as-snn_gr_0.5_lam_0.22_ema_0.92_l1_scale_on,0.150500,0.187298,0.071094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,0.92,NaN,NaN,0.5,NaN,NaN,NaN,0.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,as-snn,as-snn_gr_0.5_lam_0.2_ema_0.9_l1_scale_on,0.150619,0.132559,0.061587,400.0,False,NaN,NaN,NaN,NaN,NaN,NaN,True,0.90,NaN,NaN,0.5,NaN,NaN,NaN,0.20,NaN,input,NaN,NaN,l1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,as-snn,as-snn_gr_0.5_lam_0.2_ema_0.9_l1_scale_on,0.152216,0.146668,0.100206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,0.90,NaN,NaN,0.5,NaN,NaN,NaN,0.20,NaN,NaN,NaN,NaN,l1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,as-snn,as-snn_gr_0.5_lam_0.16_ema_0.9_l1_scale_on,0.153717,0.209743,0.084100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,0.90,NaN,NaN,0.5,NaN,NaN,NaN,0.16,NaN,NaN,NaN,NaN,l1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,as-snn,as-snn_gr_0.5_lam_0.22_ema_0.9_l1_scale_on,0.153929,0.217123,0.097460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,0.90,NaN,NaN,0.5,NaN,NaN,NaN,0.22,NaN,NaN,NaN,NaN,l1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



### ewc — 5 configs


,method_base,method,c2_final_mae,forget,co2,mp_activity_every,mp_activity_verbose,mp_anchor_batches,mp_assume_binary_spikes,mp_attach_to,mp_beta,mp_bias,mp_buffer_size,mp_do_synaptic_scaling,mp_ema,mp_fisher_batches,mp_flatten_spatial,mp_gamma_ratio,mp_habit_decay,mp_k,mp_lambd,mp_lambda_a,mp_log_every,mp_measure_at,mp_num_bins,mp_p,mp_penalty_mode,mp_replay_ratio,mp_reset_counters_each_task,mp_soft_mask_temp,mp_target_active_frac,mp_tau,mp_th_max,mp_th_min,mp_update_on_eval,mp_verbose,mp_vt_scale
5,ewc,ewc_lam_3e+06,0.170263,0.234944,0.062440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200.0,NaN,NaN,NaN,NaN,3000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ewc,ewc_lam_6e+07,0.189837,0.364852,0.121103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400.0,NaN,NaN,NaN,NaN,60000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,ewc,ewc_lam_6e+07,0.196272,0.357630,0.063638,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400.0,NaN,NaN,NaN,NaN,60000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,ewc,ewc_lam_6e+07,0.201300,0.260496,0.064367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400.0,NaN,NaN,NaN,NaN,60000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,ewc,ewc_lam_3e+08,0.212777,0.522253,0.063458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,512.0,NaN,NaN,NaN,NaN,300000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



### rehearsal — 5 configs


,method_base,method,c2_final_mae,forget,co2,mp_activity_every,mp_activity_verbose,mp_anchor_batches,mp_assume_binary_spikes,mp_attach_to,mp_beta,mp_bias,mp_buffer_size,mp_do_synaptic_scaling,mp_ema,mp_fisher_batches,mp_flatten_spatial,mp_gamma_ratio,mp_habit_decay,mp_k,mp_lambd,mp_lambda_a,mp_log_every,mp_measure_at,mp_num_bins,mp_p,mp_penalty_mode,mp_replay_ratio,mp_reset_counters_each_task,mp_soft_mask_temp,mp_target_active_frac,mp_tau,mp_th_max,mp_th_min,mp_update_on_eval,mp_verbose,mp_vt_scale
10,rehearsal,rehearsal_buf_1000_rr_15,0.155257,0.111317,0.061955,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,rehearsal,rehearsal_buf_1000_rr_15,0.155874,0.111526,0.061909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,rehearsal,rehearsal_buf_3000_rr_15,0.158088,0.136199,0.062621,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,rehearsal,rehearsal_buf_1000_rr_15,0.158157,0.141025,0.062036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,rehearsal,rehearsal_buf_3000_rr_15,0.158161,0.176255,0.062559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



### sa-snn — 5 configs


,method_base,method,c2_final_mae,forget,co2,mp_activity_every,mp_activity_verbose,mp_anchor_batches,mp_assume_binary_spikes,mp_attach_to,mp_beta,mp_bias,mp_buffer_size,mp_do_synaptic_scaling,mp_ema,mp_fisher_batches,mp_flatten_spatial,mp_gamma_ratio,mp_habit_decay,mp_k,mp_lambd,mp_lambda_a,mp_log_every,mp_measure_at,mp_num_bins,mp_p,mp_penalty_mode,mp_replay_ratio,mp_reset_counters_each_task,mp_soft_mask_temp,mp_target_active_frac,mp_tau,mp_th_max,mp_th_min,mp_update_on_eval,mp_verbose,mp_vt_scale
15,sa-snn,sa-snn,0.153774,0.164533,0.099314,NaN,NaN,NaN,False,f6,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,0.12,NaN,NaN,NaN,NaN,NaN,5000000.0,NaN,NaN,NaN,NaN,NaN,32.0,2.0,1.0,NaN,NaN,1.20
16,sa-snn,sa-snn,0.154740,0.111237,0.063431,NaN,NaN,NaN,False,f6,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,0.13,NaN,NaN,NaN,NaN,NaN,5000000.0,NaN,NaN,NaN,NaN,NaN,32.0,2.0,1.0,NaN,NaN,1.20
17,sa-snn,sa-snn,0.155247,0.171723,0.074032,NaN,NaN,NaN,False,f6,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,0.12,NaN,NaN,NaN,NaN,NaN,5000000.0,NaN,NaN,NaN,NaN,NaN,32.0,2.0,1.0,NaN,NaN,1.20
18,sa-snn,sa-snn,0.156392,0.183107,0.063243,NaN,NaN,NaN,NaN,f6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.15,NaN,NaN,NaN,NaN,NaN,2000000.0,NaN,NaN,True,NaN,NaN,28.0,NaN,NaN,NaN,NaN,1.15
19,sa-snn,sa-snn,0.156689,0.207765,0.063025,NaN,NaN,NaN,False,f6,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,0.12,NaN,NaN,NaN,NaN,NaN,2000000.0,NaN,NaN,True,NaN,NaN,28.0,2.0,1.0,False,NaN,1.25



### sca-snn — 5 configs


,method_base,method,c2_final_mae,forget,co2,mp_activity_every,mp_activity_verbose,mp_anchor_batches,mp_assume_binary_spikes,mp_attach_to,mp_beta,mp_bias,mp_buffer_size,mp_do_synaptic_scaling,mp_ema,mp_fisher_batches,mp_flatten_spatial,mp_gamma_ratio,mp_habit_decay,mp_k,mp_lambd,mp_lambda_a,mp_log_every,mp_measure_at,mp_num_bins,mp_p,mp_penalty_mode,mp_replay_ratio,mp_reset_counters_each_task,mp_soft_mask_temp,mp_target_active_frac,mp_tau,mp_th_max,mp_th_min,mp_update_on_eval,mp_verbose,mp_vt_scale
20,sca-snn,sca-snn_bins50_beta0.5_bias0_temp0.5_ab24_flat1,0.150095,0.147925,0.075241,NaN,NaN,24.0,NaN,NaN,0.50,0.00,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,NaN,NaN,NaN,NaN,0.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,sca-snn,sca-snn_bins50_beta0.52_bias0_temp0.5_ab24_flat1,0.153973,0.200748,0.063836,NaN,NaN,24.0,NaN,NaN,0.52,0.00,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,NaN,NaN,NaN,NaN,0.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,sca-snn,sca-snn_bins50_beta0.45_bias-0.05_temp0.45_ab2...,0.154628,0.205157,0.061994,NaN,NaN,24.0,NaN,f6,0.45,-0.05,NaN,NaN,NaN,NaN,False,NaN,0.99,NaN,NaN,NaN,65536.0,NaN,50.0,NaN,NaN,NaN,NaN,0.45,NaN,NaN,NaN,NaN,NaN,False,NaN
23,sca-snn,sca-snn_bins50_beta0.5_bias0_temp0.5_ab24_flat0,0.154640,0.093250,0.051350,NaN,NaN,24.0,NaN,f6,0.50,0.00,NaN,NaN,NaN,1000.0,False,NaN,0.99,NaN,1.0e9,NaN,65536.0,NaN,50.0,NaN,NaN,NaN,NaN,0.50,NaN,NaN,NaN,NaN,NaN,False,NaN
24,sca-snn,sca-snn_bins50_beta0.5_bias0_temp0.5_ab24_flat1,0.154702,0.160906,0.063764,NaN,NaN,24.0,NaN,NaN,0.50,0.00,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,NaN,NaN,NaN,NaN,0.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<a id="sec-09"></a>
## 9) Resumen de artefactos generados

**Objetivo**

Listar los CSV/PNG creados en la carpeta `SUMMARY` para tener una visión de conjunto
y poder referenciarlos desde otros notebooks (p.ej. el de análisis para la memoria).

[↑ Volver al índice](#toc)


In [38]:
print("[OK] Artifacts en:", SUMMARY)
for p in sorted(SUMMARY.glob("*.csv")) + sorted(SUMMARY.glob("*.png")):
    print(" -", p.name)

print("[LAST] Manifiesto final:", OUT / "summary" / "_last.json")

[OK] Artifacts en: /home/cesar/proyectos/TFM_SNN/outputs/summary/paperset_std_rate_2025-12-11_2042
 - pareto.csv
 - relative_to_ewc.csv
 - results_table_fromdisk.csv
 - scorecards_por_metodo_vs_ewc.csv
 - selection_table.csv
 - selection_with_params.csv
 - top5_por_metodo.csv
 - top6_composite.csv
 - winners_per_methodbase.csv
 - scatter_all.png
 - scatter_winners.png
[LAST] Manifiesto final: /home/cesar/proyectos/TFM_SNN/outputs/summary/_last.json


In [39]:
# === Selección explícita de modelos NUEVOS (recorte + más vueltas) ==========
NEW_RUNS = [
    "continual_std_as-snn_gr_0.5_lam_0.2_ema_0.9_l1_scale_on_best_as_gr050_lam020_ema090_rate_model-PilotNetSNN_66x200_gray_seed_42",
    "continual_std_sa-snn_best_sa_k013_vt120_rate_model-PilotNetSNN_66x200_gray_seed_42",
]

# === Selección explícita de modelos VIEJOS (sin recorte) ====================
OLD_RUNS = [
    "continual_std_as-snn_gr_0.5_lam_0.2_ema_0.9_l1_scale_on_std_as_input_gr050_lam0p20_scaling_on_s07_rate_model-PilotNetSNN_66x200_gray_seed_42",
    "continual_std_sa-snn_tune_sa_k013_vt120_rate_model-PilotNetSNN_66x200_gray_seed_42",
    "continual_std_sca-snn_bins50_beta0.5_bias0_temp0.5_ab24_flat0_std_sca_f6_b50_beta050_bias0_t050_ab24_rate_model-PilotNetSNN_66x200_gray_seed_42",
]

cols = [
    "run_dir",
    "method",
    "preset",
    "circuito1_best_mae",
    "circuito1_final_mae",
    "circuito2_best_mae",
    "circuito2_final_mae",
    "avg_forget_rel",
    "emissions_kg",
]

mask_new = df["run_dir"].isin(NEW_RUNS)
mask_old = df["run_dir"].isin(OLD_RUNS)

print("=== Modelos NUEVOS (recorte + más vueltas) ===")
display(
    df.loc[mask_new, cols]
      .sort_values(["method", "circuito2_best_mae"])
)

print("\n=== Modelos VIEJOS de referencia (sin recorte) ===")
display(
    df.loc[mask_old, cols]
      .sort_values(["method", "circuito2_best_mae"])
)


=== Modelos NUEVOS (recorte + más vueltas) ===


,run_dir,method,preset,circuito1_best_mae,circuito1_final_mae,circuito2_best_mae,circuito2_final_mae,avg_forget_rel,emissions_kg



=== Modelos VIEJOS de referencia (sin recorte) ===


,run_dir,method,preset,circuito1_best_mae,circuito1_final_mae,circuito2_best_mae,circuito2_final_mae,avg_forget_rel,emissions_kg
53,continual_std_as-snn_gr_0.5_lam_0.2_ema_0.9_l1...,as-snn_gr_0.5_lam_0.2_ema_0.9_l1_scale_on,std,0.119297,0.150925,0.150619,0.150619,0.132559,0.061587
4,continual_std_sa-snn_tune_sa_k013_vt120_rate_m...,sa-snn,std,0.118192,0.144487,0.154740,0.154740,0.111237,0.063431
25,continual_std_sca-snn_bins50_beta0.5_bias0_tem...,sca-snn_bins50_beta0.5_bias0_temp0.5_ab24_flat0,std,0.126052,0.149560,0.154640,0.154640,0.093250,0.051350
